# 第7章: 機械学習

本章では、[Stanford Sentiment Treebank (SST)](https://nlp.stanford.edu/sentiment/) データセットを用い、評判分析器（ポジネガ分類器）を構築する。ここでは処理を簡略化するため、[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されているSSTデータセットを用いる。


## 60. データの入手・整形

GLUEのウェブサイトから[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)データセットを取得せよ。学習データ（`train.tsv`）と検証データ（`dev.tsv`）のぞれぞれについて、ポジティブ (1) とネガティブ (0) の事例数をカウントせよ。

In [3]:
import os
import zipfile
import requests

# データセットのダウンロード
url = "https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"
zip_path = "./data/SST-2.zip"

response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("./data")

In [4]:
train_path = "./data/SST-2/train.tsv"
dev_path = "./data/SST-2/dev.tsv"

def count_labels(filepath):
    with open(filepath, "r")as f:
        lines = f.readlines()
        count_positive = 0
        count_negative = 0
        
        for line in lines[1:]:
            label = line.split("\t")[1].strip()
            if label == "0":
                count_negative += 1
            elif label == "1":
                count_positive += 1
                
        return f"Positive: {count_positive}, Negative: {count_negative}"

print("Train set => " + count_labels(train_path))
print("Dev set => " + count_labels(dev_path))

Train set => Positive: 37569, Negative: 29780
Dev set => Positive: 444, Negative: 428


## 61. 特徴ベクトル

Bag of Words (BoW) に基づき、学習データ（`train.tsv`）および検証データ（`dev.tsv`）のテキストを特徴ベクトルに変換したい。ここで、ある事例のテキストの特徴ベクトルは、テキスト中に含まれる単語（スペース区切りのトークン）の出現頻度で構成する。例えば、"too loud , too goofy"というテキストに対応する特徴ベクトルは、以下のような辞書オブジェクトで表現される。

```python
{'too': 2, 'loud': 1, ',': 1, 'goofy': 1}
```

各事例はテキスト、特徴ベクトル、ラベルを格納した辞書オブジェクトでまとめておく。例えば、先ほどの"too loud , too goofy"に対してラベル"0"（ネガティブ）が付与された事例は、以下のオブジェクトで表現される。

```python
{'text': 'too loud , too goofy', 'label': '0', 'feature': {'too': 2, 'loud': 1, ',': 1, 'goofy': 1}}
```

学習データと検証データの各事例を上記のような辞書オブジェクトに変換したうえで、学習データと検証データのそれぞれを、辞書オブジェクトのリストとして表現せよ。さらに、学習データの最初の事例について、正しく特徴ベクトルに変換できたか、目視で確認せよ。

In [5]:
import pandas as pd
from collections import Counter

def load_data(filepath):
    df = pd.read_csv(filepath, sep="\t", header=0)
    df.columns = ["text", "label"]
    df["label"] = df["label"].astype(int)
    return df

def make_BoW_dict(df_dataset):
    result = []
    for text, label in zip(df_dataset["text"], df_dataset["label"]):
        if not isinstance(text, str) or text.strip() == "":
            continue
        tokens = text.split()
        feature = dict(Counter(tokens))
        result.append({"text": text, "label": label, "feature": feature})
    return result

train_data = make_BoW_dict(load_data(train_path))
dev_data = make_BoW_dict(load_data(dev_path))

print(train_data[0])

{'text': 'hide new secretions from the parental units ', 'label': 0, 'feature': {'hide': 1, 'new': 1, 'secretions': 1, 'from': 1, 'the': 1, 'parental': 1, 'units': 1}}


## 62. 学習

61で構築した学習データの特徴ベクトルを用いて、ロジスティック回帰モデルを学習せよ。

In [15]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

# BoW辞書をベクトル化
vectorizer = DictVectorizer(sparse=True)
X_train = vectorizer.fit_transform([item["feature"] for item in train_data])
y_train = [item["label"] for item in train_data]

# モデル学習
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

## 63. 予測

学習したロジスティック回帰モデルを用い、検証データの先頭の事例のラベル（ポジネガ）を予測せよ。また、予測されたラベルが検証データで付与されていたラベルと一致しているか、確認せよ。

In [16]:
# 検証データのベクトル化
X_dev = vectorizer.transform([item["feature"] for item in dev_data])
y_dev = [item["label"] for item in dev_data]

# 先頭の事例の予測
pred_label = model.predict(X_dev[0])

print("予測ラベル:", pred_label[0])
print("正解ラベル:", y_dev[0])

予測ラベル: 1
正解ラベル: 1


## 64. 条件付き確率

学習したロジスティック回帰モデルを用い、検証データの先頭の事例を各ラベル（ポジネガ）に分類するときの条件付き確率を求めよ。

In [17]:
proba = model.predict_proba(X_dev[0])
print(proba[0])

[0.00432436 0.99567564]


## 65. テキストのポジネガの予測

与えられたテキストのポジネガを予測するプログラムを実装せよ。例えば、テキストとして"the worst movie I 've ever seen"を与え、ロジスティック回帰モデルの予測結果を確認せよ。


## 66. 混同行列の作成

学習したロジスティック回帰モデルの検証データにおける混同行列（confusion matrix）を求めよ。

## 67. 精度の計測

学習したロジスティック回帰モデルの正解率、適合率、再現率、F1スコアを、学習データおよび検証データ上で計測せよ。

## 68. 特徴量の重みの確認

学習したロジスティック回帰モデルの中で、重みの高い特徴量トップ20と、重みの低い特徴量トップ20を確認せよ。

## 69. 正則化パラメータの変更

ロジスティック回帰モデルを学習するとき、正則化の係数（ハイパーパラメータ）を調整することで、学習時の適合度合いを制御できる。正則化の係数を変化させながらロジスティック回帰モデルを学習し、検証データ上の正解率を求めよ。実験の結果は、正則化パラメータを横軸、正解率を縦軸としたグラフにまとめよ。